#Python Chess Engine Train

**libraries**

---

In [ ]:
import glob
import os
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

**connect to google drive**

---

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


**data**

---

import the chess games dataset from github

In [ ]:
!git clone https://github.com/iAmEthanMai/chess-games-dataset.git

fatal: destination path 'chess-games-dataset' already exists and is not an empty directory.


In [ ]:


path_fischer = '/content/chess-games-dataset/Data/CSV_FISCHER'
path_morphy = '/content/chess-games-dataset/Data/CSV_MORPHY'
path_capablanca = '/content/chess-games-dataset/Data/CSV_CAPABLANCA'

files_fischer = glob.glob(path_fischer + "/*.csv")
#files_morphy = glob.glob(path_morphy + "/*.csv")
#files_capablanca = glob.glob(path_capablanca + "/*.csv")


li = []

for filename in files_fischer:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

train = pd.concat(li, axis=0, ignore_index=True)

In [ ]:
train = shuffle(train)

In [ ]:
train.shape

(830376, 193)

In [ ]:
train.head()

,a1,b1,c1,d1,e1,f1,g1,h1,a2,b2,...,to_h7,to_a8,to_b8,to_c8,to_d8,to_e8,to_f8,to_g8,to_h8,good_move
410925,R,N,B,Q,K,B,None,R,P,P,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
759465,None,None,None,None,None,R,None,K,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
220626,R,N,B,Q,K,B,N,R,P,P,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
491497,R,None,None,None,None,R,K,None,P,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
265640,R,None,B,None,None,None,None,None,P,P,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False


**features**

---

In [ ]:
features = list(train.iloc[:, 0:192].columns)

In [ ]:
X = train[features]
y = train['good_move']

In [ ]:
categorical_columns = list(X.iloc[:, 0:63].columns)
numerical_columns = list(X.iloc[:, 64:192].columns)
feature_columns = []

for feature_name in categorical_columns:
  vocabulary = X[feature_name].unique()
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))


for feature_name in numerical_columns:
  feature_columns.append(tf.feature_column.numeric_column(feature_name,dtype = tf.float32))

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.
Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


**input function**

---

In [ ]:
def make_input_fn(data_df, label_df, num_epochs = 10, shuffle = True, batch_size = 32):
  def input_function():
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
    if shuffle:
      ds = ds.shuffle(1000)
    ds = ds.batch(batch_size).repeat(num_epochs)
    return ds
  return input_function

**split data into batches**

---

In [ ]:
def split_into_batches(df, batch_size=100000):
  nb_rows = len(df.index)
  intervals = []

  for i in range(0, nb_rows + 1, batch_size):
    intervals.append(i)

  if(intervals[-1] != nb_rows):
    intervals.append(nb_rows)

  batches_X = []
  batches_y = []

  for i in range(0, len(intervals) - 1):
    batches_X.append(train.iloc[intervals[i]:intervals[i + 1], :][features])
    batches_y.append(train.iloc[intervals[i]:intervals[i + 1], :]['good_move'])

  return batches_X, batches_y

In [ ]:
batches_X, batches_y = split_into_batches(train)

**model**

---

In [ ]:
linear_est = tf.estimator.LinearClassifier(feature_columns = feature_columns, model_dir='/content/gdrive/MyDrive/chess-engine/estimator')

Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.


**train model**

---

In [ ]:
input_functions = []
for df_X, df_y in zip(batches_X, batches_y):
  input_functions.append(make_input_fn(df_X, df_y))

In [ ]:
len(input_functions)

9

In [ ]:
# train the model on all the input functions
i = 1
j = 1

for input_function in input_functions:
  j += 1

print(j)

for input_function in input_functions:
  print('<======================================== NEW BATCH ========================================>')
  print('Batch: ' + str(i))
  i = i + 1
  linear_est.train(input_function)


# save the model
serving_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(
  tf.feature_column.make_parse_example_spec(feature_columns))

estimator_base_path = '/content/gdrive/MyDrive/chess-engine/estimator'
estimator_path = linear_est.export_saved_model(estimator_base_path, serving_input_fn)

10
<======================================== NEW BATCH ========================================>
Batch: 1


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use standard file utilities to get mtimes.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.


<======================================== NEW BATCH ========================================>
Batch: 2
<======================================== NEW BATCH ========================================>
Batch: 3
